# Produces dataset of project times from appended pipeline datasets ("all_quarters_merged.csv")

In [79]:
import pandas as pd
import logging
import dateutil
from dateutil import parser
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x) 

In [80]:
logging.basicConfig(level=logging.INFO)

# Actual Code

In [81]:
def convert_to_one_record_per_project(df):
    """
    Group the dataset by (apn, address) and then emit one row per (apn, address) pair.
    best_date and best_stat will be converted into arrays.
    All other attributes will be taken from the record with the most recent `best_date` attribute.
    """
    gb = df.groupby(['apn', 'address'])
    for k in gb.groups:
        group_df = gb.get_group(k)
        group_df = group_df.sort_values(['best_date', 'report_year', 'report_quarter'], ascending=[True, True, True])
        last_row = group_df.tail(1).copy()
        
        # identify building permit ID. Then fill in rest of quarters with this permit ID.
        building_permit=np.nan
        for index, row in group_df.iterrows():
            if pd.isnull(row['dbi_permit']) and not pd.isnull(building_permit):
                building_permit=building_permit
            else:
                building_permit = row['dbi_permit']
                
        #Identify completion quarter for those projects that have reached completion
        for index, row in group_df.iterrows():
            if row['best_stat'] !='CONSTRUCTION':
                comp_quarter= np.nan
                comp_year=np.nan
            elif row['best_stat'] == 'CONSTRUCTION':
                comp_quarter = row['report_quarter']
                comp_year = row['report_year']

        if pd.isnull(comp_quarter):
            pass
        if comp_quarter == 1 and comp_year == 2017:
            comp_quarter = np.nan
        elif comp_quarter == 4:
            comp_quarter = 1
            comp_year = 1+comp_year
        else:
            comp_quarter = 1+comp_quarter
            
        if comp_quarter ==1:
            comp_daymth= '01/01'
        elif comp_quarter ==2:
            comp_daymth= '04/01'
        elif comp_quarter==3:
            comp_daymth= '07/01'
        elif comp_quarter == 4:
            comp_daymth= '10/01'
        elif pd.isnull(comp_quarter):
            comp_daymth=np.nan

        if pd.isnull(comp_daymth):
            comp_date = np.nan
        else:
            comp_date = comp_daymth + "/" + str(comp_year)
        
        #Identify earliest "firstfiled" date
        firstfiled=''
        for index, row in group_df.iterrows():
            if pd.isnull(row['firstfiled']):
                continue
            else:
                if len(firstfiled) ==0:
                    firstfiled=row['firstfiled']
                else:
                    if dateutil.parser.parse(row['firstfiled']) < dateutil.parser.parse(firstfiled):
                        firstfiled=row['firstfiled']
                    else: 
                        firstfiled=firstfiled
        
        #Identify earliest best date
        earliest_BD = ''
        for index, row in group_df.iterrows():
            if pd.isnull(row['best_date']):
                continue
            else:
                if len(earliest_BD) == 0:
                    earliest_BD = row['best_date']
                else:
                    continue
        
        # Finalize first date variable (minimum of earliest best_date and firstfiled)
        if firstfiled =='' and earliest_BD !='':
            first_date = earliest_BD
        elif earliest_BD=='' and firstfiled !='':
            first_date = firstfiled
        elif firstfiled !='' and earliest_BD !='':
            first_date = min(firstfiled, earliest_BD)
            
        # initiate variables. Groups without these dates are blank for these variables.
        BP_date = ''
        con_date = ''

        #Identify first date for all status categories
        m=0
        for index, row in group_df.iterrows():
            if m == 0:
                status_previous = 'blah'
            if row['status']=='Building Permit Approved':
                if index == 0:
                    BP_date = row['best_date']
                elif index !=0:
                    if status_previous =='Building Permit Approved':
                        BP_date=BP_date
                    else:
                        BP_date = row['best_date']
            elif row['status']=='Under Construction':
                if index == 0:
                    con_date = row['best_date']
                elif index !=0:
                    if status_previous =='Under Construction':
                        con_date=con_date
                    else:
                        con_date = row['best_date']
            status_previous = row['status']
            m=m+1
            
        #Identify latest unit counts
        units = np.nan
        for index, row in group_df.iterrows():
            if pd.isnull(row['units']):
                continue
            else:
                units = row['units']
                
        #Identify latest net unit counts
        unitsnet= np.nan
        for index, row in group_df.iterrows():
            if pd.isnull(row['unitsnet']):
                continue
            else:
                unitsnet = row['unitsnet']
        
        #a few projects have construction best date after the completion date. In these cases, match the two.
        if pd.notnull(comp_date) & pd.notnull(con_date):
            if dateutil.parser.parse(comp_date) < dateutil.parser.parse(con_date):
                comp_date = con_date
                
        #Identify latest zoning designation
        for index, row in group_df.iterrows():
            zoning = row['zoning_simplified']
        
        last_row['firstfiled']=firstfiled
        last_row['dbi_permit']= building_permit
        last_row['comp_date']=comp_date
        last_row['BP_date'] = BP_date
        last_row['con_date'] = con_date
        last_row['first_date']=first_date
        last_row['latest_project_record_date'] = last_row.best_date
        last_row['first_project_record_date'] = group_df.iloc[0].best_date
        last_row['latest_project_status'] = last_row.best_stat
        last_row['units']=units
        last_row['unitsnet']=unitsnet
        last_row['zoning']=zoning

        ## Store a parseable list of all the project states and the dates those states were reported
        last_row['project_dates'] = str(tuple(group_df.best_date))
        last_row['project_statuses'] = str(tuple(group_df.best_stat))

        ## Store the project duration in days
        if not (pd.isnull(comp_date) or pd.isnull(first_date)):
            last_row['project_duration_days'] = (dateutil.parser.parse(comp_date) - dateutil.parser.parse(first_date)).days
        
        yield last_row


def main():
    df = pd.read_csv("cleaned/all_quarters_merged.csv")
    #consolidate status categories. Start with 3 for now
    def status_function(value):
        if value['best_stat']=="CONSTRUCTION":
            field = 'Under Construction'
        elif (value['best_stat']=='BP APPROVED') | (value['best_stat']=='BP ISSUED') | (value['best_stat']=='BP REINSTATED'):
            field = 'Building Permit Approved'
        else:
            field = 'Proposed'
        return field

    df['status']=df.apply(status_function, axis=1)
    
    #keep only variables we want
    list = ['address', 'apn', 'best_date', 'best_stat','status', 'firstfiled', 'report_quarter', 'report_year','units','unitsnet', 'dbi_permit', 'zoning_simplified', 'x', 'y']
    df = df[list]
    new_df = pd.concat(convert_to_one_record_per_project(df))
    logging.info("Writing output ({} rows, {} cols) to data/cleaned/all_quarters__one_record_per_project.csv".format(*new_df.shape))
    new_df.to_csv("cleaned/all_quarters__one_record_per_project.csv")
    return new_df

In [82]:
new_df = main()

/Users/RSW/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (1,2,6,7,14,15,19,23,24,25,26,28,29,30,31,32,37,38,39,42,43,44,45,46,47,52,53,55,57,59,60,63,69,71,72,73,74,75,76,77,78,79,81,85,89,91,96,97,98,99,100,101,102,103,104,106,111,114,116,117,118,119,120,122,123,124,126,129,131,139,142,144,147) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
INFO:root:Writing output (3551 rows, 25 cols) to data/cleaned/all_quarters__one_record_per_project.csv


In [83]:
#First, filter out those projects that are exclusively non-residential (defined as those without units)
new_df = new_df[new_df['units'] > 0]

In [84]:
#Next, keep only those that reached completion at some point over the time period
new_df=new_df[pd.notnull(new_df['comp_date'])]
new_df=new_df[new_df['firstfiled'] !='']

In [85]:
new_df=new_df[pd.notnull(new_df['firstfiled'])]

# Manual Data Fixes (Post Catching Mistake)

In [86]:
# fix duplicates
new_df['address_apn']=new_df['address']+new_df['apn']

for index, row in new_df.iterrows():
    if row['dbi_permit']=='':
        row['dbi_permit']==row['address_apn']

In [87]:
#Finally, filter out records with duplicate BP Ids
new_df[new_df.duplicated('dbi_permit', keep=False)]['address_apn'].unique()

array(['2101 & 2155 WEBSTER ST0629037',
       '2101 & 2155 WEBSTER STREET0629037', '2155 WEBSTER ST0629037',
       '1634 - 1690 PINE ST0647007', '1634-1690 PINE ST0647007',
       '1100 GOLDEN GATE AV0757025', '1239 TURK ST0757027',
       '1100 GOLDEN GATE AV0757028', '746 LAGUNA ST0794015',
       '555 FULTON ST0794028', '450 HAYES ST0808039',
       '450 HAYES STREET0808039', '447 - 453 LINDEN ST0818048',
       '447 LINDEN ST0818048', '443 LINDEN ST0818049',
       'MARKET OCTAVIA - PARCEL P0831023',
       'PARCEL P - MARKET OCTAVIA0831023', '4 OCTAVIA ST0855011',
       '8 OCTAVIA ST0855011', '55 LAGUNA ST0857001',
       '55 LAGUNA ST (BLDG 2)0857001', '55 LAGUNA STREET0857001',
       '218 BUCHANAN ST0857001A', '100 BUCHANAN ST0870003',
       '55 LAGUNA ST (BLDG 1)0870003', '3150 GEARY BL1066028',
       '3150 GEARY BL1066060', '3575 GEARY BL1083002',
       '3575 GEARY BL1084010', '1823 TURK ST1153020',
       '1823 TURK ST (RESIDENTIAL)1153020', '338 CARL ST1265009',
     

In [88]:
new_df[new_df['address_apn'] == '3575 GEARY BL1083002']['dbi_permit']

18830    200702224724
Name: dbi_permit, dtype: object

In [89]:
new_df[new_df['dbi_permit']=='200702224724']

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y,zoning,zoning_simplified,address_apn
18830,,3575 GEARY BL,1083002,2009-12-14,CONSTRUCTION,01/01/2010,2009-08-19,200702224724,2003-05-20,2009-08-19,2003-05-20,2009-12-14,CONSTRUCTION,"('2009-08-19', '2009-12-14')",2418.00,"('CONSTRUCTION', 'CONSTRUCTION')",4,2009,Under Construction,150.00,150.00,-122.46,37.78,NC-3,NC-3,3575 GEARY BL1083002
12727,,3575 GEARY BL,1084010,2010-09-24,CONSTRUCTION,04/01/2011,2010-04-26,200702224724,2003-05-20,2010-04-26,2003-05-20,2010-09-24,CONSTRUCTION,"('2010-04-26', '2010-05-07', '2010-08-16', '20...",2873.00,"('CONSTRUCTION', 'CONSTRUCTION', 'CONSTRUCTION...",1,2011,Under Construction,150.00,nan,-122.46,37.78,NC-3,NC-3,3575 GEARY BL1084010


# Previous Manual Fixes (Pre-Mistake)

In [90]:
# Pre Mistake Manual Fixes

# fix duplicates
new_df['address_apn']=new_df['address']+new_df['apn']

#decided that both this and its duplicate BP at 1169 Market Street are two phases of the same project. Revise start date and drop 1169 Market
new_df.loc[(new_df['address']=='1190 MISSION ST') & (new_df['apn']=='3702052'), 'first_date'] = '2003-07-15'
new_df = new_df[new_df['address_apn'] !='1169 MARKET ST3702053']

#Same project. Same dates except for construction date. Choosing best of the two dates based on Google street view.
new_df.loc[(new_df['address']=='49 MOSS ST') & (new_df['apn']=='3731119'), 'con_date'] = '2015-05-20'
new_df = new_df[new_df['address_apn'] !='47 MOSS ST3731119']

#Assume that 101 EXECUTIVE PARK BL duplicates are legitimately separate projects. Different unit counts, etc.

#Changed address name in final Q4 2015. Extend completion date by 1 quarter. Drop the duplicate.
new_df.loc[(new_df['address']=='690 MONTEREY BLVD') & (new_df['apn']=='3094055'), 'comp_date'] = '2015-05-20'
new_df=new_df[new_df['address_apn'] !='690 MONTEREY BL3094055']

#1340-1390 MISSION ST no dbi permit in any quarter just leave in

#Multiple Hunters point shipyards. I extend completion date and change units to latest available information. 
new_df.loc[(new_df['address']=='HUNTERS POINT SHIPYARD PHASE I') & (new_df['apn']=='4591C001'), 'comp_date'] = '10/01/2016'
new_df.loc[(new_df['address']=='HUNTERS POINT SHIPYARD PHASE I') & (new_df['apn']=='4591C001'), 'units'] = 167
new_df=new_df[new_df['address_apn'] !='HP SHIPYARD PHASE 1 (UNDER CONSTRUCTION)4591C001']

#Same project. Change construction date to reflect earliest information.
new_df.loc[(new_df['address']=='800 BROTHERHOOD WAY (UNDER CONSTRUCTION)') & (new_df['apn']=='7331005'), 'con_date'] = '2012-09-11'
new_df=new_df[new_df['address_apn'] !='800 BROTHERHOOD7331005']

#HUDSON AVE AND WHITNEY YOUNG CIR4711118—no dbi permit information in any quarter. Just leave in

#Same project. Change completion date to latest available info.
new_df.loc[(new_df['address']=='MARKET OCTAVIA - PARCEL P') & (new_df['apn']=='0831023'), 'comp_date'] = '01/01/2016'
new_df=new_df[new_df['address_apn'] !='PARCEL P - MARKET OCTAVIA0831023']

#duplicate projects. Update to reflect latest completion date information.
new_df=new_df[new_df['address_apn'] !='443 LINDEN ST0818049']

#duplicate projects. Update completion date and units to most recent information. Everything else to earliest info.
new_df.loc[(new_df['address']=='2101 & 2155 WEBSTER STREET') & (new_df['apn']=='0629037'), 'con_date'] = '2015-03-18'
new_df=new_df[new_df['address_apn'] !='2155 WEBSTER ST0629037']

#duplicate projects but one just pops up for one quarter so just drop it.
new_df=new_df[new_df['address_apn'] !='70 DOUGLASS ST2625032']

#duplicate projects. Update completion date to most recent. Update construction date to earliest.
new_df.loc[(new_df['address']=='3575 GEARY BL') & (new_df['apn']=='1084010'), 'con_date'] = '2010-04-26'
new_df=new_df[new_df['address_apn'] !='3575 GEARY BL1083002']

#duplicate projects. Drop ones that are out of date.
new_df=new_df[new_df['address_apn'] !='1100 GOLDEN GATE AV0757028']
new_df=new_df[new_df['address_apn'] !='1239 TURK ST0757027']

#duplicate projects. Update completion date to most recent. Update construction date to earliest available.
#Determined through PIM that 218 Buchanan alteration to 55 Laguna Street project. Update information to earliest possible.
new_df.loc[(new_df['address']=='55 LAGUNA ST') & (new_df['apn']=='0857001'), 'con_date'] = '2013-08-07'
new_df.loc[(new_df['address']=='55 LAGUNA ST') & (new_df['apn']=='0857001'), 'BP_date'] = '2013-06-11'
new_df=new_df[new_df['address_apn'] !='55 LAGUNA STREET0857001']
new_df=new_df[new_df['address_apn'] !='218 BUCHANAN ST0857001A']

#duplicate with out of date info. Drop.
new_df=new_df[new_df['address_apn'] !='782-786 ANDOVER ST5825032']

#duplicate with out of date info. Drop.
new_df=new_df[new_df['address_apn'] !='746 CAROLINA ST4096109']

#duplicate with out of date info. Drop.
new_df=new_df[new_df['address_apn'] !='1301 INDIANA ST4228010']

#145 AMES ST and 1068 GUERRERO ST seem like legitimately different projects despite duplicate building permits.

#duplicate with out of date info. Drop.
new_df=new_df[new_df['address_apn'] !='3150 GEARY BL1066060']

#seems like the same project (8 Octavia versus 4 Octavia). Updating completion date.
new_df.loc[(new_df['address']=='4 OCTAVIA ST') & (new_df['apn']=='0855011'), 'BP_date'] = '10/01/2014'
new_df=new_df[new_df['address_apn'] !='8 OCTAVIA ST0855011']

#drop duplicate 338 Carl Street. No new info.
new_df=new_df[new_df['address_apn'] !='338 CARL ST1265047']

#drop duplicate 1076 Hampshire Street and 1078 Hampshire Street. No new info.
new_df=new_df[new_df['address_apn'] !='1076 HAMPSHIRE ST4152045']
new_df=new_df[new_df['address_apn'] !='1078 HAMPSHIRE ST4152046']

#duplicates. Updating comp date to most recent info.
new_df.loc[(new_df['address']=='1420 MISSION ST') & (new_df['apn']=='3507039'), 'comp_date'] = '04/01/2015'
new_df=new_df[new_df['address_apn'] !='1400 MISSION ST3507039']

#there are 2 101 EXECUTIVE PARK BL's. I believe that they are separate projects so leaving them so here.

#duplicates. Updating comp date to most recent info. Updating construction date to earliest info.
new_df.loc[(new_df['address']=='1 ECKER ST') & (new_df['apn']=='3708022'), 'con_date'] = '2009-05-12'
new_df=new_df[new_df['address_apn'] !='1 ECKER ST3708056']


# Explore Duplicate Data

In [91]:
#Finally, filter out records with duplicate BP Ids
new_df[new_df.duplicated('dbi_permit', keep=False)]['address_apn'].unique()


array(['2101 & 2155 WEBSTER ST0629037',
       '2101 & 2155 WEBSTER STREET0629037', '1634 - 1690 PINE ST0647007',
       '1634-1690 PINE ST0647007', '746 LAGUNA ST0794015',
       '555 FULTON ST0794028', '450 HAYES ST0808039',
       '450 HAYES STREET0808039', '447 - 453 LINDEN ST0818048',
       '447 LINDEN ST0818048', '55 LAGUNA ST0857001',
       '55 LAGUNA ST (BLDG 2)0857001', '100 BUCHANAN ST0870003',
       '55 LAGUNA ST (BLDG 1)0870003', '1823 TURK ST1153020',
       '1823 TURK ST (RESIDENTIAL)1153020', '2800 SLOAT BL2515001',
       '2800 SLOAT BLVD2515001', '83 PANORAMA DR2821010',
       '83 PANORAMA DR2821023', '287 CRESTA VISTA DR2993088',
       '287 CRESTA VISTA DR2993090', '1446 OCEAN AV3197010',
       '1490 OCEAN AV3197010', '1340-1390 MISSION ST3508053',
       '145 AMES ST3632038', '1068 GUERRERO ST3632039',
       'TRINITY PLAZA PHASE III3702308', 'TRINITY PLAZA PHASE III3702391',
       '41 TEHAMA ST3736074', '57 TEHAMA ST3736078A',
       '201 FOLSOM ST3746001', '

In [92]:
new_df[new_df['address_apn'] == '2101 & 2155 WEBSTER STREET']['dbi_permit']

Series([], Name: dbi_permit, dtype: object)

In [93]:
#new_df[pd.isnull(new_df['dbi_permit'])]

In [94]:
new_df[new_df['dbi_permit']==201112070234.00]

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y,zoning,zoning_simplified,address_apn


In [95]:
new_df[new_df['address_apn']=='HUNTERS VIEW4624031']['firstfiled']

Series([], Name: firstfiled, dtype: object)

In [96]:
#explore duplicated
new_df[new_df['address']=='2155 WEBSTER ST']

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y,zoning,zoning_simplified,address_apn


In [97]:
new_df[new_df['dbi_permit']=='200705010136']

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y,zoning,zoning_simplified,address_apn
11132,2008-12-11,72 TOWNSEND ST,3789003,2015-11-04,CONSTRUCTION,01/01/2016,2013-11-18,200705010136,2007-05-01,2008-12-11,2007-05-01,2015-11-04,CONSTRUCTION,"('2008-12-11', '2008-12-11', '2008-12-11', '20...",3167.00,"('BP APPROVED', 'BP APPROVED', 'BP APPROVED', ...",4,2015,Under Construction,74.00,74.00,-122.39,37.78,SB-DTR,SB-DTR,72 TOWNSEND ST3789003


In [98]:
new_df[new_df['project_duration_days']<0]

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y,zoning,zoning_simplified,address_apn


# Manual Data Cleaning
Based off of identified duplicates in the previous section

In [99]:
#this has a duplicate, same address, different APN
new_df=new_df[(new_df['address']!='3575 GEARY BL') | (new_df['apn']=='1084010')]

In [100]:
#same project, happening at different periods
new_df.loc[(new_df['address']=='1634-1690 PINE ST') & (new_df['apn']=='0647007'), 'comp_date']='07/01/2016'
new_df=new_df[(new_df['address']!='1634 - 1690 PINE ST') | (new_df['apn']=='0647007')]

In [101]:
new_df.loc[(new_df['address']=='1634-1690 PINE ST') & (new_df['apn']=='0647007')]

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y,zoning,zoning_simplified,address_apn
5245,,1634-1690 PINE ST,0647007,2015-10-29,CONSTRUCTION,07/01/2016,2015-07-30,201312184508,2000-11-27,2012-07-17,2000-11-27,2015-10-29,CONSTRUCTION,"('2012-07-17', '2012-07-17', '2012-07-17', '20...",5421.00,"('PL FILED', 'PL FILED', 'PL FILED', 'PL FILED...",3,2015,Under Construction,260.00,260.00,-122.42,37.79,NC-3,NC-3,1634-1690 PINE ST0647007


In [102]:
#Be sure to redo project duration after all manual cleaning

In [103]:
#do median time based on neighborhood

In [104]:
#breakdown time to completion by stage for those projects that this can be done for